# Acknowledgment

This implementation is fully based on the following code:

- code: https://github.com/NielsRogge/Transformers-Tutorials/blob/master/DINOv2/Train_a_linear_classifier_on_top_of_DINOv2_for_semantic_segmentation.ipynb
- Author: NielsRogge

# Description
- Task: semantic segmentation
- Dataset: ADE20K
- Model: DINOv2-small, DINOv2-base
- Evaluation: Linear

# Installations and Imports

In [1]:
Install = False
if Install:
    !pip3 install evaluate
    !pip3 install transformers
    !pip3 install datasets
    !pip3 install albumentations

In [3]:
import torch
from torch.optim import AdamW
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import evaluate
import numpy as np
from PIL import Image
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from transformers import Dinov2Model, Dinov2PreTrainedModel
from transformers.modeling_outputs import SemanticSegmenterOutput
from datasets import load_dataset
import albumentations as A

# Functions and Classes

In [4]:
def get_dataset(dataset_dir):
    dataset = load_dataset(dataset_dir)
    print(dataset)
    return dataset

def generate_id2label(num_labels):
    id2label = {0: "label0"}

    for i in range(1, num_labels + 1):
        id2label[i] = f"label{i}"
    
    return id2label

def get_transform_linear():
    
    ADE_MEAN = np.array([123.675, 116.280, 103.530]) / 255
    ADE_STD = np.array([58.395, 57.120, 57.375]) / 255

    train_transform = A.Compose([
        A.Resize(width=448, height=448),
        A.HorizontalFlip(p=0.5),
        A.Normalize(mean=ADE_MEAN.tolist(), std=ADE_STD.tolist()),
    ])

    val_transform = A.Compose([
        A.Resize(width=448, height=448),
        A.Normalize(mean=ADE_MEAN.tolist(), std=ADE_STD.tolist()),

    ])
    
    return train_transform, val_transform

def collate_fn(inputs):
    batch = dict()
    batch["pixel_values"] = torch.stack([i[0] for i in inputs], dim=0)
    batch["labels"] = torch.stack([i[1] for i in inputs], dim=0)
    batch["original_images"] = [i[2] for i in inputs]
    batch["original_segmentation_maps"] = [i[3] for i in inputs]

    return batch

def train_model(model, train_dataloader, metric, id2label, learning_rate=1e-5, epochs=3, device=None):

    for name, param in model.named_parameters():
        if name.startswith("dinov2"):
            param.requires_grad = False
    
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    optimizer = AdamW(model.parameters(), lr=learning_rate)

    model.to(device)

    model.train()
    
    metrics_history = []

    for epoch in range(epochs):
        print("Epoch:", epoch)
        for idx, batch in enumerate(tqdm(train_dataloader)):
            pixel_values = batch["pixel_values"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(pixel_values, labels=labels)
            loss = outputs.loss

            loss.backward()
            optimizer.step()

            optimizer.zero_grad()

            with torch.no_grad():
                predicted = outputs.logits.argmax(dim=1)
                
                metric.add_batch(predictions=predicted.detach().cpu().numpy(), 
                                 references=labels.detach().cpu().numpy())

            if idx % 100 == 0:
                metrics = metric.compute(num_labels=len(id2label),
                                        ignore_index=0,
                                        reduce_labels=False)
                
                metrics_history.append({
                    'epoch': epoch,
                    'batch_idx': idx,
                    'mean_iou': metrics['mean_iou'],
                    'mean_accuracy': metrics['mean_accuracy']
                })                
                
                print("Loss:", loss.item())
                print("Mean IOU:", metrics["mean_iou"])
                print("Mean Accuracy:", metrics["mean_accuracy"])
                print("----------------------------------")
    
    return metrics_history

def val_eval_linear(model, val_dataloader, metric, id2label, device=None):

    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    model.to(device)

    model.eval()

    with torch.no_grad():
        for batch in tqdm(val_dataloader):
            pixel_values = batch["pixel_values"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(pixel_values, labels=labels)

            predicted = outputs.logits.argmax(dim=1)

            metric.add_batch(predictions=predicted.detach().cpu().numpy(), 
                             references=labels.detach().cpu().numpy())

    final_metrics = metric.compute(num_labels=len(id2label),
                                   ignore_index=0,
                                   reduce_labels=False)

    print(f"Final Mean IOU: {final_metrics['mean_iou']}")
    print(f"Final Mean Accuracy: {final_metrics['mean_accuracy']}")

In [5]:
class SegmentationDataset(Dataset):
  def __init__(self, dataset, transform, feature_img, feature_seg):
    self.dataset = dataset
    self.transform = transform
    self.feature_img = feature_img
    self.feature_seg = feature_seg

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    item = self.dataset[idx]
    original_image = np.array(item[self.feature_img])
    original_segmentation_map = np.array(item[self.feature_seg])

    transformed = self.transform(image=original_image, mask=original_segmentation_map)
    image, target = torch.tensor(transformed['image']), torch.LongTensor(transformed['mask'])
    
    if image.dim() == 2:
        image = image.unsqueeze(2)
        image = image.expand(-1, -1, 3)

    image = image.permute(2, 0, 1)

    return image, target, original_image, original_segmentation_map

class LinearClassifier(torch.nn.Module):
    def __init__(self, in_channels, tokenW=32, tokenH=32, num_labels=1):
        super(LinearClassifier, self).__init__()

        self.in_channels = in_channels
        self.width = tokenW
        self.height = tokenH
        self.classifier = torch.nn.Conv2d(in_channels, num_labels, (1,1))

    def forward(self, embeddings):
        embeddings = embeddings.reshape(-1, self.height, self.width, self.in_channels)
        embeddings = embeddings.permute(0,3,1,2)

        return self.classifier(embeddings)

class Dinov2ForSemanticSegmentationLinear(Dinov2PreTrainedModel):
  def __init__(self, config):
    super().__init__(config)

    self.dinov2 = Dinov2Model(config)
    self.classifier = LinearClassifier(config.hidden_size, 32, 32, config.num_labels)

  def forward(self, pixel_values, output_hidden_states=False, output_attentions=False, labels=None):

    outputs = self.dinov2(pixel_values,
                            output_hidden_states=output_hidden_states,
                            output_attentions=output_attentions)

    patch_embeddings = outputs.last_hidden_state[:,1:,:]

    logits = self.classifier(patch_embeddings)
    logits = torch.nn.functional.interpolate(logits, size=pixel_values.shape[2:], mode="bilinear", align_corners=False)
    # logits = torch.nn.functional.interpolate(logits, size=(512, 512), mode="bilinear", align_corners=False)
    # labels = torch.nn.functional.interpolate(labels.unsqueeze(1).float(), size=(512, 512), mode="nearest").squeeze(1).long()
      
    loss = None
    if labels is not None:

      loss_fct = torch.nn.CrossEntropyLoss(ignore_index=0)
      loss = loss_fct(logits.squeeze(), labels.squeeze())

    return SemanticSegmenterOutput(
        loss=loss,
        logits=logits,
        hidden_states=outputs.hidden_states,
        attentions=outputs.attentions,
    )

# Hyperparameters

In [ ]:
num_labels = 150
id2label = generate_id2label(num_labels)

batch_size = 32
learning_rate = 1e-5
epochs = 10

# Dataset Processing

In [8]:
dataset_dir = "sezer12138/ADE20k_Segementation"
dataset = get_dataset(dataset_dir)

DatasetDict({
    train: Dataset({
        features: ['image', 'annotated', 'Scene_category'],
        num_rows: 20210
    })
    val: Dataset({
        features: ['image', 'annotated', 'Scene_category'],
        num_rows: 2000
    })
})


In [9]:
train_set = "train"
val_set = "val"
feature_img = "image"
feature_seg = "annotated"

In [12]:
train_transform, val_transform = get_transform_linear()
train_dataset = SegmentationDataset(dataset[train_set], transform=train_transform, feature_img=feature_img, feature_seg=feature_seg)
val_dataset = SegmentationDataset(dataset[val_set], transform=val_transform, feature_img=feature_img, feature_seg=feature_seg)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Train and Evaluation

### Base Model

In [13]:
model = Dinov2ForSemanticSegmentationLinear.from_pretrained("facebook/dinov2-base", id2label=id2label, num_labels=len(id2label))

Some weights of Dinov2ForSemanticSegmentation were not initialized from the model checkpoint at facebook/dinov2-base and are newly initialized: ['classifier.classifier.bias', 'classifier.classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
metric = evaluate.load("mean_iou")
metric_val = evaluate.load("mean_iou")

In [15]:
results = train_model(model, train_dataloader, metric, id2label, learning_rate=learning_rate, epochs=epochs, device=None)

Epoch: 0


  0%|          | 0/632 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.12/site-packages/datasets/features/image.py:348: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")
/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label


Loss: 5.363834857940674
Mean IOU: 0.0005900922703899486
Mean Accuracy: 0.0029101335194810576
----------------------------------
Loss: 4.920153617858887
Mean IOU: 0.002362893366769786
Mean Accuracy: 0.007310141630805983
----------------------------------
Loss: 4.555122375488281
Mean IOU: 0.005520635373931384
Mean Accuracy: 0.012564513693235165
----------------------------------
Loss: 4.225812911987305
Mean IOU: 0.010951113118287177
Mean Accuracy: 0.02150667986881082
----------------------------------
Loss: 4.047341346740723
Mean IOU: 0.018638176633005505
Mean Accuracy: 0.03550549933241794
----------------------------------
Loss: 3.690267324447632
Mean IOU: 0.026897144546919636
Mean Accuracy: 0.049713538865356025
----------------------------------
Loss: 3.485060930252075
Mean IOU: 0.034872895123228886
Mean Accuracy: 0.06306722349672932
----------------------------------
Epoch: 1


  0%|          | 0/632 [00:00<?, ?it/s]

...

Loss: 0.9536737203598022
Mean IOU: 0.28061698201115104
Mean Accuracy: 0.3550393307430293
----------------------------------
Loss: 1.1119657754898071
Mean IOU: 0.2776653717412066
Mean Accuracy: 0.3543513208319342
----------------------------------
Loss: 1.2184925079345703
Mean IOU: 0.2860223019137122
Mean Accuracy: 0.3599998262283748
----------------------------------
Loss: 0.9511573314666748
Mean IOU: 0.2910153166488449
Mean Accuracy: 0.3686636180294783
----------------------------------
Loss: 1.094383955001831
Mean IOU: 0.2886520454565116
Mean Accuracy: 0.36322896868624277
----------------------------------
Loss: 1.0588369369506836
Mean IOU: 0.2899247889480869
Mean Accuracy: 0.3700658324490427
----------------------------------
Loss: 1.1562860012054443
Mean IOU: 0.297796682658697
Mean Accuracy: 0.37684267619139394
----------------------------------
Epoch: 8


  0%|          | 0/632 [00:00<?, ?it/s]

Loss: 1.3313068151474
Mean IOU: 0.29759164245088054
Mean Accuracy: 0.38131162254783846
----------------------------------
Loss: 0.9416850209236145
Mean IOU: 0.3062503178596032
Mean Accuracy: 0.38481621682894646
----------------------------------
Loss: 1.104833722114563
Mean IOU: 0.30566513290325525
Mean Accuracy: 0.3875390531002355
----------------------------------
Loss: 0.912018358707428
Mean IOU: 0.31179708841844944
Mean Accuracy: 0.39527154595051667
----------------------------------
Loss: 0.9954028725624084
Mean IOU: 0.3137648140294118
Mean Accuracy: 0.3956226726057431
----------------------------------
Loss: 1.047498345375061
Mean IOU: 0.3140219725558593
Mean Accuracy: 0.3946988210703095
----------------------------------
Loss: 1.0393638610839844
Mean IOU: 0.32365100812598163
Mean Accuracy: 0.40600233431079413
----------------------------------
Epoch: 9


  0%|          | 0/632 [00:00<?, ?it/s]

Loss: 0.9707915186882019
Mean IOU: 0.3091414436544581
Mean Accuracy: 0.3894899555326953
----------------------------------
Loss: 0.9607212543487549
Mean IOU: 0.3243567954323603
Mean Accuracy: 0.41127065602702717
----------------------------------
Loss: 1.0408984422683716
Mean IOU: 0.32590131221118196
Mean Accuracy: 0.41124665475134
----------------------------------
Loss: 0.8639803528785706
Mean IOU: 0.32981538747296163
Mean Accuracy: 0.4119614725402649
----------------------------------
Loss: 1.0694427490234375
Mean IOU: 0.33121582730426363
Mean Accuracy: 0.4181904736450819
----------------------------------
Loss: 1.007149338722229
Mean IOU: 0.32643699443829144
Mean Accuracy: 0.4142692304902199
----------------------------------
Loss: 0.8094237446784973
Mean IOU: 0.33718877333686076
Mean Accuracy: 0.42218690108834445
----------------------------------


In [16]:
val_eval_linear(model, val_dataloader, metric_val, id2label, device=None)

  0%|          | 0/63 [00:00<?, ?it/s]

Final Mean IOU: 0.32945676686083786
Final Mean Accuracy: 0.42086342137961696


In [17]:
with open("./seg_ade20k_base_linear.txt", "w") as file:
    for item in results:
        file.write(f"{item}\n")

### Small Model

In [13]:
model = Dinov2ForSemanticSegmentationLinear.from_pretrained("facebook/dinov2-small", id2label=id2label, num_labels=len(id2label))

config.json:   0%|          | 0.00/331 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/88.2M [00:00<?, ?B/s]

Some weights of Dinov2ForSemanticSegmentation were not initialized from the model checkpoint at facebook/dinov2-small and are newly initialized: ['classifier.classifier.bias', 'classifier.classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
metric = evaluate.load("mean_iou")
metric_val = evaluate.load("mean_iou")

In [15]:
results = train_model(model, train_dataloader, metric, id2label, learning_rate=learning_rate, epochs=epochs, device=None)

Epoch: 0


  0%|          | 0/632 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.12/site-packages/datasets/features/image.py:348: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")
/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0/mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  iou = total_area_intersect / total_area_union
/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label


Loss: 5.444570541381836
Mean IOU: 0.0014819876380996722
Mean Accuracy: 0.012857570863623592
----------------------------------
Loss: 5.113446235656738
Mean IOU: 0.0016824489387340204
Mean Accuracy: 0.006822135484356362
----------------------------------
Loss: 4.8038225173950195
Mean IOU: 0.003437326357317041
Mean Accuracy: 0.010671583950915118
----------------------------------
Loss: 4.524355411529541
Mean IOU: 0.006840760775584131
Mean Accuracy: 0.017425402435347983
----------------------------------
Loss: 4.266073226928711
Mean IOU: 0.012070405010766571
Mean Accuracy: 0.027422808145991493
----------------------------------
Loss: 4.060877323150635
Mean IOU: 0.017973868870558497
Mean Accuracy: 0.03858380892613026
----------------------------------
Loss: 3.852776050567627
Mean IOU: 0.0253439444931257
Mean Accuracy: 0.0510660611334291
----------------------------------
Epoch: 1


  0%|          | 0/632 [00:00<?, ?it/s]

...

Loss: 1.0704164505004883
Mean IOU: 0.20313608456362284
Mean Accuracy: 0.26714191281561406
----------------------------------
Loss: 1.183836817741394
Mean IOU: 0.20846130715851766
Mean Accuracy: 0.26812101844751673
----------------------------------
Loss: 1.223578691482544
Mean IOU: 0.2108795980734892
Mean Accuracy: 0.27040526855450886
----------------------------------
Loss: 0.8744267225265503
Mean IOU: 0.21558845416048422
Mean Accuracy: 0.2770215077898305
----------------------------------
Loss: 1.113162875175476
Mean IOU: 0.22095378427411602
Mean Accuracy: 0.2833707873891312
----------------------------------
Loss: 1.1184031963348389
Mean IOU: 0.2255190186679063
Mean Accuracy: 0.28709441928128604
----------------------------------
Loss: 1.1269102096557617
Mean IOU: 0.22480304314323857
Mean Accuracy: 0.2876003805319568
----------------------------------
Epoch: 8


  0%|          | 0/632 [00:00<?, ?it/s]

Loss: 1.0237345695495605
Mean IOU: 0.2216905728093489
Mean Accuracy: 0.28890533034458393
----------------------------------
Loss: 1.0657769441604614
Mean IOU: 0.23606174062747246
Mean Accuracy: 0.30092695168919414
----------------------------------
Loss: 1.261810302734375
Mean IOU: 0.23163398735294674
Mean Accuracy: 0.29409598019740946
----------------------------------
Loss: 1.0676225423812866
Mean IOU: 0.2406690965708772
Mean Accuracy: 0.303504020336182
----------------------------------
Loss: 0.9197268486022949
Mean IOU: 0.2478972412893729
Mean Accuracy: 0.3149676175794545
----------------------------------
Loss: 0.9636109471321106
Mean IOU: 0.24119785699492183
Mean Accuracy: 0.30773507614195966
----------------------------------
Loss: 1.153996229171753
Mean IOU: 0.2473988210155824
Mean Accuracy: 0.315238123058237
----------------------------------
Epoch: 9


  0%|          | 0/632 [00:00<?, ?it/s]

Loss: 1.1155385971069336
Mean IOU: 0.24726141967973092
Mean Accuracy: 0.31388393563334815
----------------------------------
Loss: 1.107743501663208
Mean IOU: 0.25809128873338133
Mean Accuracy: 0.32494892783406754
----------------------------------
Loss: 0.9834823608398438
Mean IOU: 0.2505465622053893
Mean Accuracy: 0.31782294683199097
----------------------------------
Loss: 0.9801182150840759
Mean IOU: 0.2535666756416481
Mean Accuracy: 0.3210864829280693
----------------------------------
Loss: 1.0059361457824707
Mean IOU: 0.26677432838781706
Mean Accuracy: 0.3380139053873112
----------------------------------
Loss: 1.0620789527893066
Mean IOU: 0.2659471059435727
Mean Accuracy: 0.33667294538225495
----------------------------------
Loss: 1.0350384712219238
Mean IOU: 0.27143782965731106
Mean Accuracy: 0.34279011373907425
----------------------------------


In [16]:
val_eval_linear(model, val_dataloader, metric_val, id2label, device=None)

  0%|          | 0/63 [00:00<?, ?it/s]

Final Mean IOU: 0.2645381000943276
Final Mean Accuracy: 0.3366199306649156


In [17]:
with open("./seg_ade20k_small_linear.txt", "w") as file:
    for item in results:
        file.write(f"{item}\n")